# Merge datasets

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import geopandas as gpd


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import geopandas as gpd
import requests
import os

In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Data Cleaning")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "9g") 
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/11 09:17:17 WARN Utils: Your hostname, DESKTOP-702MS12 resolves to a loopback address: 127.0.1.1; using 172.17.210.38 instead (on interface eth0)
24/09/11 09:17:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/11 09:17:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Join merchant & consumer data

In [3]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Data Merge")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "9g") 
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

24/09/10 16:03:54 WARN Utils: Your hostname, LAPTOP-406UJ3L3 resolves to a loopback address: 127.0.1.1; using 172.22.104.23 instead (on interface eth0)
24/09/10 16:03:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/10 16:03:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/10 16:03:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Join merchant & consumer data

In [4]:
def join_func(left: DataFrame, right: DataFrame, left_keys: ArrayType, 
              method: StringType='inner', right_keys: ArrayType=None, drop_left: BooleanType=False) -> DataFrame:
    """
    Joins two pyspark dataframes on the left and right specified keys, returning the inner join by default.
    Identifies records not joined in left and right datasets.
    """
    
    if not right_keys:
        right_keys = left_keys

    join_cond = "&".join([f"(left.{i} == right.{j})" for i,j in zip(left_keys, right_keys)])

    if not drop_left:
        drop_cols = [right[c] for c in right_keys]
    else: 
        drop_cols = [left[c] for c in left_keys]

    # Join
    joined_df = left.join(right, eval(join_cond), how=method).drop(*drop_cols)

    # See which return all the rows from left df that do not have a match in the right df
    mismatch = left.join(right, eval(join_cond), how='left_anti').drop(*drop_cols)
    print(f"Number of unjoined left records: {mismatch.count()}")

    return joined_df

## Use clean dataset

In [7]:
# Load in merchant data (csv)
merchant = spark.read.parquet("../data/curated/part_1/tbl_merchants.parquet")

# Load in merchant fraud (csv)
merchant_fp = pd.read_csv("../data/tables/part_1/merchant_fraud_probability.csv")
merchant_fp = spark.createDataFrame(merchant_fp)

# Load in consumer list (csv)
consumer = pd.read_csv("../data/tables/part_1/tbl_consumer.csv", delimiter="|")
consumer = spark.createDataFrame(consumer)

# Load in consumer fraud (csv)
consumer_fp = pd.read_csv("../data/tables/part_1/consumer_fraud_probability.csv")
consumer_fp = spark.createDataFrame(consumer_fp)

consumer_ud = spark.read.parquet("../data/tables/part_1/consumer_user_details.parquet")

**Join merchants data**

`tbl_merchant` to `merchant_fraud_probability`

In [8]:
merchant_full = join_func(merchant, merchant_fp, ['merchant_abn'], 'left', drop_left=False)
merchant_full.orderBy('fraud_probability', ascending=False).limit(5)

Number of unjoined left records: 3978
+--------------------+------------+--------------------+-------------+---------+--------------+-----------------+
|                name|merchant_abn|               goods|revenue_level|take_rate|order_datetime|fraud_probability|
+--------------------+------------+--------------------+-------------+---------+--------------+-----------------+
|        Mollis Corp.| 12516851436|watch, clock, and...|            a|     6.71|          NULL|             NULL|
|     Ante Industries| 15613631617|motor vehicle sup...|            e|     0.35|          NULL|             NULL|
|Pellentesque Habi...| 19839532017|cable, satellite,...|            b|     4.94|          NULL|             NULL|
|Mauris Nulla Inte...| 34440496342|opticians, optica...|            c|     2.85|          NULL|             NULL|
|Quis Tristique Ac...| 35344855546|watch, clock, and...|            c|     2.92|          NULL|             NULL|
+--------------------+------------+---------------

In [10]:
merchant_full.orderBy('fraud_probability', ascending=False).limit(5)

name,merchant_abn,goods,revenue_level,take_rate,order_datetime,fraud_probability
Tempus Mauris Ltd,35575706403,antique shops - s...,b,3.46,2022-02-20,91.09606847149963
Ut Corporation,97884414539,antique shops - s...,a,6.82,2021-10-19,89.79919971536573
Duis At Inc.,14530561097,"jewelry, watch, c...",c,1.69,2021-09-15,80.80054474543395
Ut Industries,18737319630,"computers, comput...",a,5.7,2021-09-25,72.73069736562613
Mi Eleifend Egest...,85482742429,"gift, card, novel...",b,3.91,2021-11-27,70.88131110541714


**Join customer data**

`tbl_consumer` to `consumer_user_detail` and `consumer_fraud_probability`

In [20]:
consumer_full_ids = join_func(consumer, consumer_ud, ['consumer_id'], 'left') # match user_id to consumers
consumer_full = join_func(consumer_full_ids, consumer_fp, ['user_id'], 'left') # match fraud probaibility to consumers

# Drop irrelevant columns
consumer_full = consumer_full.drop(*['address'])
consumer_full.limit(10)

24/09/10 16:17:13 WARN TaskSetManager: Stage 141 contains a task of very large size (1709 KiB). The maximum recommended task size is 1000 KiB.


Number of unjoined left records: 0


24/09/10 16:17:14 WARN TaskSetManager: Stage 148 contains a task of very large size (1709 KiB). The maximum recommended task size is 1000 KiB.


Number of unjoined left records: 479871


24/09/10 16:17:15 WARN TaskSetManager: Stage 155 contains a task of very large size (1709 KiB). The maximum recommended task size is 1000 KiB.
24/09/10 16:17:15 WARN TaskSetManager: Stage 160 contains a task of very large size (1709 KiB). The maximum recommended task size is 1000 KiB.


name,state,postcode,gender,consumer_id,user_id,order_datetime,fraud_probability
Yolanda Williams,WA,6935,Female,1195503,1,2022-02-20,9.80543113652096
Mary Smith,NSW,2782,Female,179208,2,2021-08-30,9.599513915425788
Mary Smith,NSW,2782,Female,179208,2,2021-09-25,10.069850934775245
Jill Jones MD,NT,862,Female,1194530,3,2021-11-03,8.300636455314633
Lindsay Jimenez,NSW,2780,Female,154128,4,2021-10-09,9.63330241109042
Rebecca Blanchard,WA,6355,Female,712975,5,2022-02-08,9.02022421158597
Rebecca Blanchard,WA,6355,Female,712975,5,2021-10-04,10.868364868449886
Rebecca Blanchard,WA,6355,Female,712975,5,2022-01-11,27.496186536467164
Karen Chapman,NSW,2033,Female,407340,6,2021-12-12,10.459280127078758
Andrea Jones,QLD,4606,Female,511685,7,NULL,NULL


**Join customers and transaction data**

In [14]:
# Read transaction dataset
transaction1 = spark.read.parquet("../data/tables/part_2")
transaction2 = spark.read.parquet("../data/tables/part_3")
transaction3 = spark.read.parquet("../data/tables/part_4")

transaction = transaction1.union(transaction2).union(transaction3)

In [22]:
# Join customers to transactions
consumer_transaction = consumer_full_ids.join(transaction, on=['user_id'])

# Count number of customers in dataset with no transactions
mismatch_c_t = consumer_full_ids.join(transaction, on='user_id', how='left_anti')
print(f"There are {mismatch_c_t.count()} customers with no matching transactions")

24/09/10 16:18:03 WARN TaskSetManager: Stage 176 contains a task of very large size (1709 KiB). The maximum recommended task size is 1000 KiB.


There are 475918 customers with no matching transactions


In [23]:
consumer_transaction.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- consumer_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



In [24]:
consumer_transaction.limit(5)

24/09/10 16:18:30 WARN TaskSetManager: Stage 186 contains a task of very large size (1709 KiB). The maximum recommended task size is 1000 KiB.
24/09/10 16:18:47 WARN TaskSetManager: Stage 192 contains a task of very large size (1709 KiB). The maximum recommended task size is 1000 KiB.


user_id,name,address,state,postcode,gender,consumer_id,merchant_abn,dollar_value,order_id,order_datetime
7,Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685,33064796871,373.0873675184212,fe188788-b89f-4dd...,2021-08-20
7,Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685,68435002949,232.5364986739752,b4a89891-a113-45e...,2021-08-20
7,Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685,41944909975,30.910755230234322,302ae628-8eba-4a5...,2021-08-20
7,Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685,21439773999,91.18655746114226,4524fdc9-73f0-477...,2021-08-21
7,Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685,86662713230,38.8137172956379,28f9e0f3-858d-445...,2021-08-19


# Join external datasets

In [27]:
medians = pd.read_csv("../data/curated/sa2_dataset/C21_G02_SA2_clean.csv")

In [33]:
# Column names
variables = {1: "median_age", 
             2: "median_total_personal_income",
             3: "median_total_family_income",
             4: "median_total_household_income",
             5: "median_mortgage_repayment",
             6: "median_rent",
             7: "avg_people_per_bedroom",
             8: "avg_household_size"}

medians = medians.pivot(index='sa2_code', columns=['type_of_value_code'], values='obs_value').reset_index().rename(columns=variables)
medians.columns.name = None
medians['sa2_code'] = medians.sa2_code.astype(str)

medians.head(5)

,sa2_code,median_age,median_total_personal_income,median_total_family_income,median_total_household_income,median_mortgage_repayment,median_rent,avg_people_per_bedroom,avg_household_size
0,101021007,51.0,760.0,1886.0,1429.0,1732.0,330.0,0.8,2.2
1,101021008,38.0,975.0,2334.0,1989.0,1950.0,350.0,0.8,2.6
2,101021009,37.0,996.0,2233.0,1703.0,1700.0,330.0,0.9,2.1
3,101021010,36.0,1104.0,2412.0,1796.0,1700.0,310.0,0.9,2.1
4,101021012,37.0,1357.0,3332.0,3014.0,2300.0,430.0,0.8,2.9


In [49]:
# Read in zone file
sa2_boundary_gdf = gpd.read_file("../data/curated/sa2_boundary/SA2_2021_AUST_GDA2020_clean.shp")
sa2_names = sa2_boundary_gdf[['sa2_code21', 'sa2_name21']].astype('string')

In [53]:
# Find records with null statistics to identify SA2 zones with null median/average values
null_regions = medians[medians.isna().any(axis=1)]
null_regions = null_regions.merge(sa2_names, left_on='sa2_code', right_on='sa2_code21')
null_regions.iloc[:,-2:]

,sa2_code21,sa2_name21


In [ ]:
# Download data
url = """https://github.com/matthewproctor/australianpostcodes/blob/92a036281ee4009be03cca3ab0b8b1a49b21dca7/australian_postcodes.csv"""
base_path = "../data/tables/poa_dataset"

response = requests.get(url, headers={'accept': 'text/csv'}, stream=True)
response.raise_for_status()

# Create directory if it doesn't exist
if not os.path.exists(base_path):
    os.makedirs(base_path)

with open("../data/tables/poa_dataset/postcodes_to_sa2.csv", 'wb') as file:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            file.write(chunk)

In [ ]:
# get sa2 - postcodes
# write code to come up with useful statistics per 